# Efficient Yelp API Calls (Core)

by Israel Diaz

## Assignment

* For this assignment, you will be working with the Yelp API.

* As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

* Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

* Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

## Solution

### Imports

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### API setup

In [2]:
with open('C:/Users/diazi/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define parameters of the search

In [3]:
# set our API call parameters
LOCATION = 'SA, TX'
TERM = 'Burger'

In [4]:
JSON_FILE = "data/results_in_progress_SA_Burger.json"
JSON_FILE

'data/results_in_progress_STGO_Barros_Luco.json'

### Define JSON file function

In [5]:
#@Create Json File Function
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE) ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")


    ## If it does NOT exist:
    else:

        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

### Create JSON file and check for results

In [6]:
#@Running file and call api

## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                                offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
print(f'Number search pages: {n_pages}')

[!] data/results_in_progress_STGO_Barros_Luco.json already exists. Deleting previous file...
[i] data/results_in_progress_STGO_Barros_Luco.json not found. Saving empty list to new file.
- 0 previous results found.
Number search pages: 85


### Saving searches into JSON file

In [7]:
#@Saving results into JSON file

for i in tqdm_notebook( range(1,n_pages+1)):

    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)

    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)



    ## append new results and save to file
    previous_results.extend(results['businesses'])

    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

    time.sleep(.2)

  0%|          | 0/85 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


It stopped at 50th call

In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,1pR8Nw9DAZSnMdpQbM5VRw,fattboy-burgers-and-dogs-san-antonio,Fattboy Burgers & Dogs,https://s3-media2.fl.yelpcdn.com/bphoto/tDtsh_...,False,https://www.yelp.com/biz/fattboy-burgers-and-d...,705,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 29.49462, 'longitude': -98.5356}",[delivery],$$,"{'address1': '2345 Vance Jackson Rd', 'address...",+12103773288,(210) 377-3288,2051.741303
1,_SSmQGFUykYBwUM2elSnbw,the-good-news-burgers-san-antonio-2,The Good News Burgers,https://s3-media2.fl.yelpcdn.com/bphoto/_HLbdh...,False,https://www.yelp.com/biz/the-good-news-burgers...,113,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 29.505741, 'longitude': -98.498459}",[delivery],$$,"{'address1': '6900 San Pedro', 'address2': 'St...",+12107819171,(210) 781-9171,1978.987257
2,y0G19V0kCxK3XDE4PuEjKw,tycoon-flats-san-antonio-2,Tycoon Flats,https://s3-media1.fl.yelpcdn.com/bphoto/_7HK_w...,False,https://www.yelp.com/biz/tycoon-flats-san-anto...,1217,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 29.452280000000002, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '2926 N Saint Marys St', 'address...",+12103200819,(210) 320-0819,5494.398854
3,RoyklFAqrt_D4IyGnBMNFw,mr-juicy-san-antonio-2,Mr Juicy,https://s3-media4.fl.yelpcdn.com/bphoto/UVN2PF...,False,https://www.yelp.com/biz/mr-juicy-san-antonio-...,422,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,"{'latitude': 29.4662, 'longitude': -98.4995}",[delivery],$$,"{'address1': '3315 San Pedro', 'address2': '',...",+12109949838,(210) 994-9838,3451.037489
4,PcDvDnbLGMcNXYtcmP_TfQ,burger-factory-live-oak,Burger Factory,https://s3-media1.fl.yelpcdn.com/bphoto/zdXNfe...,False,https://www.yelp.com/biz/burger-factory-live-o...,8,"[{'alias': 'burgers', 'title': 'Burgers'}]",5.0,"{'latitude': 29.57761606342342, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '7525 N Loop 1604 E', 'address2':...",+12105304444,(210) 530-4444,19169.035479


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,8nS6jpx95HL9yZe11WXang,culichis-and-beer-tx-san-antonio,Culichis and Beer Tx,https://s3-media2.fl.yelpcdn.com/bphoto/g1ZJeO...,False,https://www.yelp.com/biz/culichis-and-beer-tx-...,34,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 29.50379, 'longitude': -98.58572}",[delivery],NaN,"{'address1': '7271 Wurzbach Rd', 'address2': '...",+12108881525,(210) 888-1525,6979.819100
996,H7Q0PUWc_HAl_35oe8lsqA,la-taqueria-de-monterrey-san-antonio-2,La Taqueria de Monterrey,https://s3-media3.fl.yelpcdn.com/bphoto/banfPy...,False,https://www.yelp.com/biz/la-taqueria-de-monter...,161,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 29.484182, 'longitude': -98.559871}","[pickup, delivery]",$,"{'address1': '2715 Hillcrest Dr', 'address2': ...",+12109948686,(210) 994-8686,4546.061486
997,sOwjQkPLaSYEpqBeCLHgLw,spice-fine-indian-cuisine-san-antonio-3,Spice Fine Indian Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/FEMn5B...,False,https://www.yelp.com/biz/spice-fine-indian-cui...,482,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.0,"{'latitude': 29.4925527, 'longitude': -98.5553...","[pickup, delivery]",$$,"{'address1': '3720 NW Loop 410', 'address2': N...",+12102539658,(210) 253-9658,3965.641102
998,0JmsNrzin3MB5Su0Df9oZA,sonic-drive-in-san-antonio-84,Sonic Drive-In,https://s3-media3.fl.yelpcdn.com/bphoto/y3ppOR...,False,https://www.yelp.com/biz/sonic-drive-in-san-an...,18,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.5,"{'latitude': 29.43080369913767, 'longitude': -...",[],NaN,"{'address1': '225 Talley Rd', 'address2': '', ...",+12102651050,(210) 265-1050,23733.217222
999,M1kmzMkFgOATfnULmzadSw,andgravy-123-foodtruck-san-antonio,AndGravy 123 Foodtruck,https://s3-media1.fl.yelpcdn.com/bphoto/VakA6W...,False,https://www.yelp.com/biz/andgravy-123-foodtruc...,7,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.0,"{'latitude': 29.661439453373, 'longitude': -98...",[],NaN,"{'address1': '', 'address2': '', 'address3': N...",+12103945065,(210) 394-5065,23632.580946


### Looking for duplicated

In [9]:
# check for duplicate ID's
final_df.duplicated(subset='id').sum()

0

No duplicated values

### Saving dataframe into CSV file

In [10]:
# save the final results to a compressed csv
final_df.to_csv('data/final_results_SA_Burger.csv.gz', compression='gzip',index=False)